In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import time


In [2]:
class KMeansScratch:
    def __init__(self, n_clusters, distance="euclidean", max_iter=500, random_state=42):
        self.n_clusters = n_clusters
        self.distance_name = distance
        self.max_iter = max_iter
        self.random_state = np.random.RandomState(random_state)
        self.centroids = None

    def _distance(self, a, b):
        if self.distance_name == "euclidean":
            diff = a - b
            return np.sqrt(np.dot(diff, diff))
        elif self.distance_name == "cosine":
            num = np.dot(a, b)
            den = np.linalg.norm(a) * np.linalg.norm(b)
            if den == 0:
                return 1.0
            cos = num / den
            return 1.0 - cos
        elif self.distance_name == "jaccard":
            a_pos = np.clip(a, 0, None)
            b_pos = np.clip(b, 0, None)
            num = np.minimum(a_pos, b_pos).sum()
            den = np.maximum(a_pos, b_pos).sum()
            if den == 0:
                return 0.0
            sim = num / den
            return 1.0 - sim
        else:
            raise ValueError("Unknown distance")

    def _init_centroids(self, X):
        n_samples = X.shape[0]
        idx = self.random_state.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = X[idx].copy()

    def _assign_clusters(self, X):
        n_samples = X.shape[0]
        labels = np.empty(n_samples, dtype=int)
        sse = 0.0
        for i in range(n_samples):
            x = X[i]
            best = None
            best_dist = None
            for c in range(self.n_clusters):
                d = self._distance(x, self.centroids[c])
                if best is None or d < best_dist:
                    best = c
                    best_dist = d
            labels[i] = best
            sse += best_dist ** 2
        return labels, sse

    def _update_centroids(self, X, labels):
        n_features = X.shape[1]
        new_centroids = np.zeros((self.n_clusters, n_features), dtype=float)
        counts = np.zeros(self.n_clusters, dtype=int)
        for x, label in zip(X, labels):
            new_centroids[label] += x
            counts[label] += 1
        for k in range(self.n_clusters):
            if counts[k] > 0:
                new_centroids[k] /= counts[k]
            else:
                idx = self.random_state.randint(0, X.shape[0])
                new_centroids[k] = X[idx]
        return new_centroids

    def fit(self, X, stop_rule="combined"):
        X = np.asarray(X, dtype=float)
        self._init_centroids(X)
        history_sse = []
        prev_sse = None
        iterations = 0
        for it in range(self.max_iter):
            labels, sse = self._assign_clusters(X)
            history_sse.append(sse)
            new_centroids = self._update_centroids(X, labels)
            stop_centroid = np.allclose(self.centroids, new_centroids)
            stop_sse_increase = prev_sse is not None and sse > prev_sse
            prev_sse = sse
            self.centroids = new_centroids
            iterations = it + 1
            if stop_rule == "centroid" and stop_centroid:
                break
            if stop_rule == "sse_increase" and stop_sse_increase:
                break
            if stop_rule == "combined" and (stop_centroid or stop_sse_increase):
                break
        labels, final_sse = self._assign_clusters(X)
        return labels, final_sse, history_sse, iterations


In [3]:
def majority_vote_labels(y_true, cluster_labels, k):
    y_true = np.asarray(y_true)
    cluster_labels = np.asarray(cluster_labels)
    mapping = {}
    for c in range(k):
        mask = cluster_labels == c
        if not np.any(mask):
            mapping[c] = None
        else:
            values = y_true[mask]
            most_common = Counter(values).most_common(1)[0][0]
            mapping[c] = most_common
    y_pred = np.empty_like(y_true)
    for i, c in enumerate(cluster_labels):
        if mapping[c] is None:
            y_pred[i] = -1
        else:
            y_pred[i] = mapping[c]
    return y_pred, mapping


def accuracy_score(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_pred != -1
    if not np.any(mask):
        return 0.0
    return (y_true[mask] == y_pred[mask]).mean()


def run_single_experiment(X, y, k, distance_name, stop_rule, max_iter):
    model = KMeansScratch(n_clusters=k, distance=distance_name, max_iter=max_iter)
    t0 = time.time()
    cluster_labels, sse, sse_history, iterations = model.fit(X, stop_rule=stop_rule)
    runtime = time.time() - t0
    y_pred, mapping = majority_vote_labels(y, cluster_labels, k)
    acc = accuracy_score(y, y_pred)
    result = {
        "distance": distance_name,
        "stop_rule": stop_rule,
        "sse": float(sse),
        "accuracy": float(acc),
        "iterations": int(iterations),
        "time": float(runtime),
        "mapping": mapping,
    }
    return result


In [5]:
data = pd.read_csv("D:\datasets\kmeans_data\data.csv")

if "y" in data.columns:
    y = data["y"].values
    X = data.drop(columns=["y"]).values
else:
    y = data.iloc[:, -1].values
    X = data.iloc[:, :-1].values

k = len(np.unique(y))
distances = ["euclidean", "cosine", "jaccard"]
stop_rules = ["combined", "centroid", "sse_increase", "max_iter"]
max_iter = 500

results = []

for stop_rule in stop_rules:
    print("=== Stop rule:", stop_rule, "===")
    for d in distances:
        r = run_single_experiment(X, y, k, d, stop_rule, max_iter)
        results.append(r)
        print(
            f"{d:9s} | SSE: {r['sse']:.4f} | Acc: {r['accuracy']:.4f} | "
            f"Iters: {r['iterations']:4d} | Time: {r['time']:.4f}s"
        )
    print()


=== Stop rule: combined ===
euclidean | SSE: 34361687572.7277 | Acc: 1.0000 | Iters:    2 | Time: 0.4566s
cosine    | SSE: 1378.4001 | Acc: 1.0000 | Iters:    2 | Time: 0.8405s
jaccard   | SSE: 5290.6207 | Acc: 1.0000 | Iters:    2 | Time: 2.4274s

=== Stop rule: centroid ===
euclidean | SSE: 34361687572.7277 | Acc: 1.0000 | Iters:    2 | Time: 0.4260s
cosine    | SSE: 1378.4001 | Acc: 1.0000 | Iters:    2 | Time: 0.7710s
jaccard   | SSE: 5290.6207 | Acc: 1.0000 | Iters:    2 | Time: 2.5992s

=== Stop rule: sse_increase ===
euclidean | SSE: 34361687572.7277 | Acc: 1.0000 | Iters:  500 | Time: 68.0465s
cosine    | SSE: 1378.4001 | Acc: 1.0000 | Iters:  500 | Time: 130.1308s
jaccard   | SSE: 5290.6207 | Acc: 1.0000 | Iters:  500 | Time: 398.8427s

=== Stop rule: max_iter ===
euclidean | SSE: 34361687572.7277 | Acc: 1.0000 | Iters:  500 | Time: 67.3342s
cosine    | SSE: 1378.4001 | Acc: 1.0000 | Iters:  500 | Time: 129.4423s
jaccard   | SSE: 5290.6207 | Acc: 1.0000 | Iters:  500 | Time: 3

In [6]:
df_results = pd.DataFrame(results)
df_results










,distance,stop_rule,sse,accuracy,iterations,time,mapping
0,euclidean,combined,3.436169e+10,1.0,2,0.456642,{0: 0}
1,cosine,combined,1.378400e+03,1.0,2,0.840520,{0: 0}
2,jaccard,combined,5.290621e+03,1.0,2,2.427369,{0: 0}
3,euclidean,centroid,3.436169e+10,1.0,2,0.425959,{0: 0}
4,cosine,centroid,1.378400e+03,1.0,2,0.770967,{0: 0}
5,jaccard,centroid,5.290621e+03,1.0,2,2.599174,{0: 0}
6,euclidean,sse_increase,3.436169e+10,1.0,500,68.046532,{0: 0}
7,cosine,sse_increase,1.378400e+03,1.0,500,130.130779,{0: 0}
8,jaccard,sse_increase,5.290621e+03,1.0,500,398.842732,{0: 0}
9,euclidean,max_iter,3.436169e+10,1.0,500,67.334226,{0: 0}


In [7]:
def summarize_results(df):
    summary = {}
    subset = df[df["stop_rule"] == "combined"]
    best_sse = subset.loc[subset["sse"].idxmin()]
    best_acc = subset.loc[subset["accuracy"].idxmax()]
    longest_time = subset.loc[subset["time"].idxmax()]
    summary["Q1_best_sse"] = best_sse["distance"]
    summary["Q2_best_accuracy"] = best_acc["distance"]
    summary["Q3_longest_time"] = longest_time["distance"]
    print("Q1 → Lowest SSE:", summary["Q1_best_sse"])
    print("Q2 → Highest Accuracy:", summary["Q2_best_accuracy"])
    print("Q3 → Most Time to Converge:", summary["Q3_longest_time"])
    print("\nSSE under different stop rules:")
    print(df.pivot_table(values="sse", index="distance", columns="stop_rule"))
    return summary

summary = summarize_results(df_results)


Q1 → Lowest SSE: cosine
Q2 → Highest Accuracy: euclidean
Q3 → Most Time to Converge: jaccard

SSE under different stop rules:
stop_rule      centroid      combined      max_iter  sse_increase
distance                                                         
cosine     1.378400e+03  1.378400e+03  1.378400e+03  1.378400e+03
euclidean  3.436169e+10  3.436169e+10  3.436169e+10  3.436169e+10
jaccard    5.290621e+03  5.290621e+03  5.290621e+03  5.290621e+03
